# Code: A sense of uncertainty

In [159]:
import numpy as np
from scipy.spatial import Delaunay
from scipy.interpolate import griddata
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import nbimporter
from the_blog import multivariate_gaussian

In [160]:
# Load and prepare data
data = np.load("../data/resnet18_cifar10.npy")
x, y, loss, acc = data[:, 0], data[:, 1], data[:, 2], data[:, 3]

points2D = np.vstack([x, y]).T
tri = Delaunay(points2D)
simplices = tri.simplices

xi = np.linspace(-1, 1, 100)
yi = np.linspace(-1, 1, 100)
loss_contours = griddata((x, y), loss, (xi[None, :], yi[:, None]), method="cubic")
acc_contours = griddata((x, y), acc, (xi[None, :], yi[:, None]), method="cubic")

In [161]:
# Loss contour lines
loss_data = go.Contour(x=xi,
                       y=yi,
                       z=loss_contours,
                       colorscale="Viridis",
                       contours=dict(showlabels=True,
                                     labelfont=dict(size=12)),
                       contours_coloring='lines',
                       line_width=3,
                       showscale=False,
                       hovertemplate="<b>Location</b>: %{x:.2f}, %{y:.2f}<br>"+
                                     "<b>Loss</b>: %{z:.2f}<extra></extra>",
                       hoverlabel=dict(bgcolor='darkslategray'))

fig = go.Figure(loss_data)

fig.update_layout(template="plotly_white",
                  xaxis=dict(constrain="domain"),
                  yaxis=dict(scaleanchor='x'),
                  hoverlabel=dict(font_size=18),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  font=dict(family="Courier New, monospace",
                            size=16,
                            color="#7f7f7f"))

In [162]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/loss2d.html',
               full_html=False,
               include_plotlyjs='cdn')

In [163]:
# Loss landscape
loss3d = ff.create_trisurf(x=x, y=y, z=loss,
                           colormap="Viridis",
                           simplices=simplices,
                           plot_edges=False,
                           show_colorbar=False).data[0]
fig = go.Figure(loss3d)
fig.update_traces(hovertemplate="<b>Location</b>: %{x:.2f}, %{y:.2f}<br>"+
                                "<b>Loss</b>: %{z:.2f}<extra></extra>",
                  hoverlabel=dict(bgcolor='darkslategray'))


# Viewpoint
camera = dict(
    eye=dict(x=0.9, y=0.9, z=1.2),
    center=dict(x=0, y=0, z=0.2))

# Figure properties
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera)

In [164]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/loss3d.html',
               full_html=False,
               include_plotlyjs='cdn')

In [173]:
# Neg. exp loss
loss3d = ff.create_trisurf(x=x, y=y, z=np.exp(-loss),
                           colormap="Viridis",
                           simplices=simplices,
                           plot_edges=False,
                           show_colorbar=False).data[0]
loss3d.hovertemplate = "<b>Location</b>: %{x:.2f}, %{y:.2f}<br><b>Neg. Loss</b>: %{z:.2f}<extra></extra>"
loss3d.hoverlabel =dict(bgcolor="darkslategray")

# 2D Gaussian
N = 50
X, Y = np.meshgrid(np.linspace(-1, 1, N), np.linspace(-1, 1, N))

mu = np.array([0., 0.])
Sigma = np.array([[ 0.19 , 0.], [0.,  0.12]])

pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

Z = multivariate_gaussian(pos, mu, Sigma)

gauss = go.Surface(x=X, y=Y, z=Z,
                   colorscale="Viridis",
                   hovertemplate="<b>Location</b>: %{x:.2f}, %{y:.2f}<br>"+
                                 "<b>Density</b>: %{z:.2f}<extra></extra>",
                   hoverlabel=dict(bgcolor="darkslategray"),
                   showscale=False)

# Figure
fig = make_subplots(rows=1,
                    cols=2,
                    horizontal_spacing=0,
                    specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.add_trace(loss3d, row=1, col=1)
fig.add_trace(gauss, row=1, col=2)

# Viewpoint
zoom = 1.3
camera = dict(
    eye=dict(x=zoom, y=zoom, z=zoom),
    center=dict(x=0, y=0, z=-0.4))

# Figure properties
fig.layout.scene1.camera = camera
fig.layout.scene2.camera = camera
fig.update_layout(scene1=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False)),
                  scene2=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False)),
                  height=400,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))

#fig = go.FigureWidget(fig)
#def cam_change(layout, camera):
#    fig.layout.scene2.camera = camera
#
#fig.layout.scene1.on_change(cam_change, 'camera')
#fig

In [174]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/loss_vs_gauss.html',
               full_html=False,
               include_plotlyjs='cdn')

In [175]:
# Accuracy contour lines
acc_data = go.Contour(x=xi,
                      y=yi,
                      z=acc_contours,
                      contours=dict(showlabels=True,
                                    labelfont=dict(size=12)),
                      contours_coloring='lines',
                      line_width=3,
                      showscale=False,
                      hovertemplate="<b>Location</b>: %{x:.2f}, %{y:.2f}<br>"+
                                    "<b>Accuracy</b>: %{z:.2f}<extra></extra>",
                      hoverlabel=dict(bgcolor='darkslategray'))


fig = go.Figure(acc_data)
fig.update_layout(template="plotly_white",
                  xaxis=dict(constrain="domain"),
                  yaxis=dict(scaleanchor='x'),
                  hoverlabel=dict(font_size=18),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  font=dict(family="Courier New, monospace",
                            size=16,
                            color="#7f7f7f"))

In [176]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/acc2d.html',
               full_html=False,
               include_plotlyjs='cdn')

In [177]:
# Accuracy with loss overlay
loss_data.line = dict(dash="dash", width=2)
loss_data.hovertemplate="<b>Location</b>: %{x:.2f}, %{y:.2f}<extra></extra>"
acc_data.hovertemplate="<b>Location</b>: %{x:.2f}, %{y:.2f}<extra></extra>"
loss_data.hoverlabel=dict(bgcolor='darkslategray')
acc_data.hoverlabel=dict(bgcolor='darkslategray')

fig = go.Figure(data=[acc_data, loss_data])
fig.update_layout(template="plotly_white",
                  xaxis=dict(constrain="domain"),
                  yaxis=dict(scaleanchor='x'),
                  hoverlabel=dict(font_size=18),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  font=dict(family="Courier New, monospace",
                            size=16,
                            color="#7f7f7f"))

In [178]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/lossacc2d.html',
               full_html=False,
               include_plotlyjs='cdn')